In [1]:
import sys
import time
import random
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import heapq

In [47]:
class Individual(object):   
   
    def __init__(self, chromosome):
            self.chromosome = chromosome[:]
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
             
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   

    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1)
        ind2 = Individual(chromosome2)
        return [ind1, ind2] 
  

    def mutate_position(self): 
        """       Bit flip
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = random.randrange(0,len(mutated_chromosome))
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome)    
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome[:])
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome)    


## Poblacion tipo Float

In [48]:
def display(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([population[i].chromosome,population[i].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

In [4]:

def init_population(pop_number, rst):
    population = []
    state_length=2
    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(pop_number):
        new_chromosome = [2*rst*random.uniform(0, 1)-rst for j in range(state_length)]
        random.shuffle(new_chromosome)
        population.append(Individual(new_chromosome))
    return population


## Selecciona los padres mediante operadores: Ruleta/ Torneo

###  Ruleta

In [13]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

### Torneo

In [6]:
def select_parent_torneo(population,size_torneo):
    
    # Escoje el primer padre
    list_indiv=[]
    x1 = np.random.permutation(len(population) )
    y1= x1[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y1[i]].fitness)
    
    iParent1=np.argmax(list_indiv)
    
    # Escoje el segundo padre, desconsiderando el primer padre   
    x2 = np.delete(x1, iParent1)
    x2 = np.random.permutation(x2)
    list_indiv=[]
    y2= x2[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y2[i]].fitness)
    iParent2=np.argmax(list_indiv)
    
    return (population[x1[iParent1]],population[x2[iParent2]])

## Iniciando una poblacion

In [29]:
rst = 5.12  # Para restringir la poblacion entre [-rst,rst] 
num_individuals=7

# Inicializa una poblacion inicial de forma aleatoria
population = init_population(num_individuals, rst)

In [30]:

display(population) #Imprime la primera poblacion 

,Poblacion,fitness
0,"[-1.1799119389708985, -1.8017206713043858]",-1
1,"[4.301656336450924, 3.4272035684664237]",-1
2,"[2.336316879016052, 1.9337857767719644]",-1
3,"[-1.9752717454785649, -2.9238337744121066]",-1
4,"[-0.9511918060109279, -1.9520411226902228]",-1
5,"[1.7428858460269554, -1.906158456792041]",-1
6,"[-1.7122359941587955, -4.658559264524199]",-1


 ## Probando los Operadores

In [31]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=19
display(population)

,Poblacion,fitness
0,"[-1.1799119389708985, -1.8017206713043858]",2
1,"[4.301656336450924, 3.4272035684664237]",23
2,"[2.336316879016052, 1.9337857767719644]",11
3,"[-1.9752717454785649, -2.9238337744121066]",17
4,"[-0.9511918060109279, -1.9520411226902228]",22
5,"[1.7428858460269554, -1.906158456792041]",5
6,"[-1.7122359941587955, -4.658559264524199]",19


- Llamando a torneo

In [32]:
size_torneo = 3

winner1,winner2=select_parent_torneo(population,size_torneo)
print(winner1.chromosome,winner1.fitness)
print(winner2.chromosome,winner2.fitness)


[4.301656336450924, 3.4272035684664237] 23
[-0.9511918060109279, -1.9520411226902228] 22


- Llamando a ruleta

In [33]:
winner1,winner2=select_parents_roulette(population)
print(winner1.chromosome,winner1.fitness)
print(winner2.chromosome,winner2.fitness)

[1.7428858460269554, -1.906158456792041] 5
[-0.9511918060109279, -1.9520411226902228] 22


## Probando Cruzamiento

In [34]:
popsize = len(population)
## Selecciona las parejas de padres para cruzamiento 
mating_pool = []
for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 

In [41]:
# Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
offspring_population = []
for i in range(len(mating_pool)): 
    offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) )
    #offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )

In [51]:
display(offspring_population)

,Poblacion,fitness
0,"[1, 0]",-1
1,"[1, -1.7122359941587955]",-1
2,"[0, 3.4272035684664237]",-1
3,"[1.9337857767719644, 1]",-1
4,"[3.4272035684664237, 0]",-1
5,"[1, 1]",-1


## Probando Mutacion

In [52]:
pmut=0.8
## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
for i in range(len(offspring_population)):
    if random.uniform(0, 1) < pmut: 
       # offspring_population[i] = offspring_population[i].mutate_swap()
       offspring_population[i] = offspring_population[i].mutate_position()


In [53]:
display(offspring_population)

,Poblacion,fitness
0,"[1, 0]",-1
1,"[1, 0]",-1
2,"[0, 3.4272035684664237]",-1
3,"[1.9337857767719644, 1]",-1
4,"[3.4272035684664237, 0]",-1
5,"[1, 1]",-1
